# YOLO Train & Inference

## YOLOv5 모델 불러오는법

* https://github.com/ultralytics/yolov5
* https://www.ultralytics.com

In [ ]:
# git clone
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# git clone 해온 yolov5 폴더로 이동
%cd yolov5

# yolov5 실행에 필요한 requirements 설치
%pip install -qr requirements.txt

### Inference

In [ ]:
# 필요한 라이브러리 import
import torch
from IPython.display import Image, clear_output

In [ ]:
!ls data/images

In [ ]:
# bus 이미지 출력
Image(filename='/kaggle/working/yolov5/data/images/bus.jpg', width=600)

In [ ]:
# zidane 이미지 출력
Image(filename='/kaggle/working/yolov5/data/images/zidane.jpg', width=600)

### `detect.py`: YOLOv5 모델을 이용한 Inference

In [ ]:
# detect.py 사용
!python detect.py --weights yolov5s.pt --img 640 --source data/images/

```

--weight : yolov5 내 어떤 모델을 활용할 지 지정

--img : 이미지 사이즈

--source : 파일 경로

```

In [ ]:
# bus 이미지에서 detect.py 를 진행한 결과 출력
Image(filename='/kaggle/working/yolov5/runs/detect/exp/bus.jpg', width=600)

In [ ]:
# zidane 이미지에서 detect.py 를 진행한 결과 출력
Image(filename='/kaggle/working/yolov5/runs/detect/exp/zidane.jpg', width=600)

## Pothole dataset 기반 Training

### 2-1. 데이터셋 다운로드

[Roboflow](https://roboflow.com/)는 컴퓨터 비전 모델 학습에 활용할 수 있는 데이터와 모델 라벨링/배포 플랫폼입니다.

[Pothole dataset](https://public.roboflow.com/object-detection/pothole) 링크에서 데이터를 가져옵니다. 여기서 받은 데이터를 `.../yolov5/pothole`에 저장합니다.

In [ ]:
%mkdir /kaggle/working/yolov5/pothole

In [ ]:
%cd /kaggle/working/yolov5/pothole

Download -> Terminal -> YOLOv5 pytorch format  선택

In [ ]:
# 코드 실행
!curl -L "https://public.roboflow.com/ds/554OsOsfOv?key=iVs10sN1Ht" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from glob import glob

# 다운로드한 데이터를 train, vaild, test 각각의 변수에 분할
train_img_list = glob('/kaggle/working/yolov5/pothole/train/images/*.jpg')
vaild_img_list = glob('/kaggle/working/yolov5/pothole/valid/images/*.jpg')
test_img_list = glob('/kaggle/working/yolov5/pothole/test/images/*.jpg')

# 데이터가 각각의 변수 안에 잘 들어갔는지 확인
print(len(train_img_list), len(test_img_list), len(vaild_img_list))

In [ ]:
import yaml

# txt 파일 경로
with open('/kaggle/working/yolov5/pothole/train.txt', 'w') as f:
    # 이미지 파일 경로
    f.write('\n'.join(train_img_list) + '\n')

# txt 파일 경로
with open('/kaggle/working/yolov5/pothole/test.txt', 'w') as f:
    # 이미지 파일 경로
    f.write('\n'.join(test_img_list) + '\n')

# txt 파일 경로
with open('/kaggle/working/yolov5/pothole/vaild.txt', 'w') as f:
    # 이미지 파일 경로
    f.write('\n'.join(vaild_img_list) + '\n')

Roboflow에 있던 pothole 데이터셋을 다운로드 받을 때 함께 다운로드 된 data.yaml 파일을 보도록 하겠습니다.

In [ ]:
%cat /kaggle/working/yolov5/pothole/data.yaml

Roboflow에서 활용되었던 경로와 현재 파일의 경로가 다르기 때문에 일치시켜주는 과정이 필요합니다. 양식을 수정하는 코드는 아래와 같습니다.

In [ ]:
with open('/kaggle/working/yolov5/pothole/data.yaml', 'w') as f:
    f.write('train: /kaggle/working/yolov5/pothole/train/images\n')
    f.write('test: /kaggle/working/yolov5/pothole/test/images\n')
    f.write('val: /kaggle/working/yolov5/pothole/valid/images\n')
    f.write('nc: 1\n')
    f.write("names: ['pothole']\n")

In [ ]:
# YAML 파일 변경 내역 확인
%cat /kaggle/working/yolov5/pothole/data.yaml

### YOLOv5 모델 config 설정

YOLOv5 모델을 사용할 때에는 모델 학습 과정에서 활용할 `data.yaml`과 모델 파일을 확인해주어야 합니다.

이ㅎ `data.yaml`에 제공된 특징(클래스 등)에 맞게 사용할 모델 구성 파일 `yolov5s.yaml`을 수정해야 합니다.

In [ ]:
import yaml

with open('/kaggle/working/yolov5/pothole/data.yaml', 'r') as stream:
    num_calsses = str(yaml.safe_load(stream)['nc'])

%cat /kaggle/working/yolov5/models/yolov5s.yaml

```
# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

```

위에 있는 `nc: 80` 을 `nc: {num_classes}` 로 수정해보도록 하겠습니다.

In [ ]:
# 파일 작성
with open('/kaggle/working/yolov5/models/custom_yolo5s.yaml', 'w') as f:
    f.write("# Parameters\n")
    f.write(f"nc: {{num_classes}}  # number of classes\n")
    f.write("depth_multiple: 0.33  # model depth multiple\n")
    f.write("width_multiple: 0.50  # layer channel multiple\n")
    f.write("anchors:\n")
    f.write("  - [10,13, 16,30, 33,23]  # P3/8\n")
    f.write("  - [30,61, 62,45, 59,119]  # P4/16\n")
    f.write("  - [116,90, 156,198, 373,326]  # P5/32\n\n")

    f.write("# YOLOv5 v6.0 backbone\n")
    f.write("backbone:\n")
    f.write("  # [from, number, module, args]\n")
    f.write("  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2\n")
    f.write("   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4\n")
    f.write("   [-1, 3, C3, [128]],\n")
    f.write("   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8\n")
    f.write("   [-1, 6, C3, [256]],\n")
    f.write("   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16\n")
    f.write("   [-1, 9, C3, [512]],\n")
    f.write("   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32\n")
    f.write("   [-1, 3, C3, [1024]],\n")
    f.write("   [-1, 1, SPPF, [1024, 5]],  # 9\n")
    f.write("  ]\n\n")

    f.write("# YOLOv5 v6.0 head\n")
    f.write("head:\n")
    f.write("  [[-1, 1, Conv, [512, 1, 1]],\n")
    f.write("   [-1, 1, nn.Upsample, [None, 2, 'nearest']],\n")
    f.write("   [[-1, 6], 1, Concat, [1]],  # cat backbone P4\n")
    f.write("   [-1, 3, C3, [512, False]],  # 13\n\n")

    f.write("   [-1, 1, Conv, [256, 1, 1]],\n")
    f.write("   [-1, 1, nn.Upsample, [None, 2, 'nearest']],\n")
    f.write("   [[-1, 4], 1, Concat, [1]],  # cat backbone P3\n")
    f.write("   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)\n\n")

    f.write("   [-1, 1, Conv, [256, 3, 2]],\n")
    f.write("   [[-1, 14], 1, Concat, [1]],  # cat head P4\n")
    f.write("   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)\n\n")

    f.write("   [-1, 1, Conv, [512, 3, 2]],\n")
    f.write("   [[-1, 10], 1, Concat, [1]],  # cat head P5\n")
    f.write("   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)\n\n")

    f.write("   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)\n")
    f.write("  ]\n")


In [ ]:
# 수정 결과 확인
%cat /kaggle/working/yolov5/models/custom_yolo5s.yaml

### 2-3. YOLOv5 모델 학습 (Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

```
# 모델 학습
!python train.py --hyperparameter arg
```

In [ ]:
# yolov5 로 위치 변경
%cd /kaggle/working/yolov5/

# 모델 학습
!python train.py --img 640 --batch 32 --epochs 100 --data ./pothole/data.yaml --cfg ./models/custom_yolo5s.yaml --weights '' --name pothole_results --cache

모델의 학습 결과는 `/yolov5/runs/train/pothole_results/` 폴더에 저장

In [ ]:
!ls /kaggle/working/yolov5/runs/train/pothole_results/

In [ ]:
Image(filename='/kaggle/working/yolov5/runs/train/pothole_results/results.png', width=1000)

In [ ]:
Image(filename='/kaggle/working/yolov5/runs/train/pothole_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='/kaggle/working/yolov5/runs/train/pothole_results/val_batch1_labels.jpg', width=1000)

### YOLOv5 Validation

모델 학습 과정에서 생성된 `best.pt` 파일을 가져오고, data, img, iou 를 재지정하여 Validation 수행을 진행합니다.

In [ ]:
# validation dataset 기반 validation
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --iou 0.65

In [ ]:
# test dataset 기반 validation
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --task test

### YOLOv5 모델 Inference

`best.pt` 에 img size, 활용할 test 데이터 경로를 지정하여 Inference를 수행합니다.

```
!python detect.py --weights runs/train/pothole_results/weights/best.pt --img 640 --source ./pothole/test/images
```

In [ ]:
%ls runs/train/pothole_results/weights

In [ ]:
!python detect.py --weights runs/train/pothole_results/weights/best.pt --img 640 --source ./pothole/test/images

결과는 `/detect/exp` 경로에 결과가 저장됩니다.

In [ ]:
import glob
import random
import IPython
from IPython.display import Image, display

image_name = random.choice(glob.glob('/kaggle/working/yolov5/runs/detect/exp3/*.jpg'))

display(Image(filename=image_name))